![head.png](figures/head.jpg)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

In [7]:
# Packages used in this notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Object oriented programming

[Object-Oriented Programming (OOP)](https://en.wikipedia.org/wiki/Object-oriented_programming) is a coding paradigm that represents real-world entities as "objects" in programming. These objects have attributes (data) and behavior (methods), which can be manipulated to solve complex problems. Python, with its clear syntax and powerful in-built tools, is a popular language for implementing OOP.

See:
* https://docs.python.org/3/tutorial/classes.html

In the context of financial economics, OOP can be particularly useful due to its abstraction and modularity. Financial instruments, for instance, can be modeled as objects. A bond could be an object with attributes such as coupon rate, face value, maturity date and methods to calculate present value, yield to maturity, and risk measures.

A class, which is a blueprint for creating objects, could be designed for a bond. From this class, you can create multiple bond objects, each representing a specific bond in the market.

Here's a basic example:

In [8]:
class Bond:
    def __init__(self, face_value, coupon_rate, maturity_date):
        self.face_value = face_value
        self.coupon_rate = coupon_rate
        self.maturity_date = maturity_date

    def calculate_present_value(self, discount_rate):
        # Implementation here
        pass

# Creating a bond object
bond1 = Bond(1000, 0.05, '2030-12-31')

Inheritance, another key feature of OOP, allows classes to inherit attributes and methods from other classes. This can represent financial concepts like hierarchies of asset classes. For example, a Bond class could inherit from a more general Security class, as all bonds are securities but have additional unique properties.

Through encapsulation, OOP allows data and methods to be bundled together, keeping code clean, reusable, and easy to maintain. Polymorphism, another crucial concept in OOP, allows methods to perform differently based on the object calling them. This is useful when dealing with a variety of financial instruments that might share similar methods but require different implementations. For instance, the method to calculate yield might differ between a bond and a stock, despite the shared name.

Let's extend the earlier example with inheritance and polymorphism:

In [9]:
class Security:  # This is a class
    def __init__(self, ticker):
        self.ticker = ticker
    
    def calculate_present_value(self):
        raise NotImplementedError("Subclasses must implement this method")

class Bond(Security):  # This is now a subclass of Security
    def __init__(self, ticker, coupon_rate, maturity_date):
        super().__init__(ticker)  # This invokes the init of the upstream class
        self.coupon_rate = coupon_rate
        self.maturity_date = maturity_date

    def calculate_present_value(self):
        # Implementation specific to bond
        pass

    def calculate_duration(self):
        # Implementation specific to bond
        pass

class Stock(Security):  # This is another subclass
    def __init__(self, ticker, dividends):
        super().__init__(ticker)  # This invokes the init of the upstream class
        self.dividends = dividends

    def calculate_present_value(self):
        # Implementation specific to stock
        pass

    def calculate_dividend_yield(self):
        # Implementation specific to stock
        pass

# Creating objects
bond1 = Bond(1000, 0.05, '2030-12-31')
stock1 = Stock(50, 2)

# Calculating yield
bond1_pv = bond1.calculate_present_value()
stock1_pv = stock1.calculate_present_value()

# Returning attributes, note the missing ()
bond1_coupon = bond1.coupon_rate

# Loose coupling

Loose coupling is a design principle in programming that advocates for minimal interdependencies between modules or components. This design makes individual modules more reusable and changes in one module less likely to impact others. In Python, and specifically in the context of financial economics, loose coupling can be achieved through various strategies, including object-oriented programming, interfaces, dependency injection, and the use of events or callbacks.

Suppose we have a trading system consisting of multiple components, such as a market data handler, a trading strategy, and an execution handler. In a tightly-coupled system, these components would directly interact with each other, leading to high dependency. Any change in one component could necessitate changes in others. However, with loose coupling, we aim to minimize these dependencies.


## Loose coupling using `class` objects

Let's demonstrate loose coupling in the example of a trading strategy:

In [10]:
class MarketDataHandler:
    def get_price(self, ticker):
        # Returns market price for the given ticker
        pass

class ExecutionHandler:
    def execute_order(self, ticker, quantity, order_type):
        # Executes the order in the market
        pass

class TradingStrategy:
    def __init__(self, data_handler: MarketDataHandler, execution_handler: ExecutionHandler):
        self.data_handler = data_handler
        self.execution_handler = execution_handler

    def execute_trade(self, ticker):
        price = self.data_handler.get_price(ticker)
        if self.is_favorable_price(price):
            self.execution_handler.execute_order(ticker, 100, 'buy')

    def is_favorable_price(self, price):
        # Decides whether the price is favorable for buying (boolean)
        pass

In this example, TradingStrategy doesn't directly depend on the specific implementations of MarketDataHandler or ExecutionHandler. Instead, it depends on the interfaces, i.e., the methods these classes are supposed to implement. This is a simple example of Dependency Injection, where the dependencies (MarketDataHandler and ExecutionHandler) are "injected" into the dependent class (TradingStrategy).

This makes the system loosely coupled as we can switch out the market data source or the execution mechanism without modifying the TradingStrategy class. For instance, you could replace a real-time market data handler with a historical one for backtesting purposes, or replace a simulated execution handler with a live one for actual trading, all without changing the trading strategy code.

## Loose coupling using a data base

Loose coupling can also be achieved with a central data base that connects code components. This is particularly useful when managing data access, running programs on different servers or in different frequencies. Please note that this type of coupling might require simultaneous read/write operations that require some form of [concurrency control](https://en.wikipedia.org/wiki/Concurrency_control). Robust concurrency for larger projects often requires a server based solution, such as [Amazon Relational Database Service (AWS-RDS)](https://aws.amazon.com/rds/).

Consider a modification of the example above, where we couple the core functions through some "database_connection":

In [11]:
class MarketDataHandler:
    def __init__(self, database_connection):
        self.database_connection = database_connection

    def get_data(self, ticker):
        # Returns market price for the given ticker
        pass

    def save_market_data(self, ticker):
        # Saves prices to database
        pass

class TradingStrategy:
    def __init__(self, database_connection):
        self.database_connection = database_connection

    def load_market_data(self, ticker):
        # Load prices 
        pass

    def save_trade_signal(self, ticker):
        # Saves trade signal for ticker to database 
        pass

class ExecutionHandler:
    def __init__(self, database_connection):
        self.database_connection = database_connection
        
    def execute_order(self, ticker, quantity, order_type):
        # Trade the order in the market
        pass
    
    def load_trade_signal(self):
        # Load trading signal from db 
        pass
    
    def execute_trade_signal(self, ticker):
        # Execute trade signal for ticker
        if self.load_trade_signal(ticker):
            self.execute_order(ticker, 100, 'buy')


In this example we have separated three essential tasks, namely:
* download market data,
* generate a trading signal, and
* execute the signal,

that connect through a central database. We will pick this up again below, with examples on the collection and storage of data.

# Market data APIs

Application Programming Interfaces (APIs) allow different software applications to communicate and share data, offering a structured way to extract data from online resources. Here, we will demonstrate how to retrieve public market data from Deribit's API using Python. You could think of APIs as a tool that allows us to couple external software and data into our workflow.

Deribit is a cryptocurrency exchange platform that offers a public API for accessing its market data. This data includes information about the current and historic state of the market, trades, and more.

In this guide, we'll be making use of Python's requests library, a simple yet powerful HTTP library. Other market data providers might come with their individual python packages, such as Refinitiv Eikon or Bloomberg. Many APIs require authentication as they are payed services or link to an individual user. An example for the latter is Deribit's trading API, which allows us to place trades after authentication.

See:
* https://docs.deribit.com/

In [12]:
import requests
import json

# Define the URL of the endpoint
url = "https://www.deribit.com/api/v2/public/get_book_summary_by_currency"

# Define the parameters
params = {
    "currency": "BTC",  # Cryptocurrency to fetch data for
    "kind": "future"    # Type of data to fetch
}

# Send the GET request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the response to JSON
    print(json.dumps(data, indent=4))  # Print the data
else:
    print(f"Failed to retrieve data: {response.status_code}")


{
    "jsonrpc": "2.0",
    "result": [
        {
            "volume_usd": 11430970.0,
            "volume_notional": 11430970.0,
            "volume": 422.40403338,
            "quote_currency": "USD",
            "price_change": -2.223,
            "open_interest": 153555650,
            "mid_price": 26723.75,
            "mark_price": 26723.38,
            "low": 26662.5,
            "last": 26720.0,
            "instrument_name": "BTC-30JUN23",
            "high": 27440.0,
            "estimated_delivery_price": 26684.55,
            "creation_timestamp": 1684935327872,
            "bid_price": 26722.5,
            "base_currency": "BTC",
            "ask_price": 26725.0
        },
        {
            "volume_usd": 165030.0,
            "volume_notional": 165030.0,
            "volume": 6.04955301,
            "quote_currency": "USD",
            "price_change": -2.1465,
            "open_interest": 18571910,
            "mid_price": 27041.25,
            "mark_price": 27038.31,

This script fetches the summary of the BTC future book from Deribit's API. The response data is then parsed to JSON and printed. For details on the data see: https://docs.deribit.com/#public-get_book_summary_by_currency

Remember to always handle API responses properly, as they may not always be successful. In this example, we check if the status code is 200 (indicating success), before proceeding to handle the data. If the status code is not 200, an error message is printed instead.

To work with the data, we can use a familiar Python package:

In [13]:
market_data = pd.DataFrame(data["result"])
display(market_data)

,volume_usd,volume_notional,volume,quote_currency,price_change,open_interest,mid_price,mark_price,low,last,instrument_name,high,estimated_delivery_price,creation_timestamp,bid_price,base_currency,ask_price,funding_8h,current_funding
0,11430970.0,11430970.0,422.404033,USD,-2.2230,153555650,26723.75,26723.38,26662.5,26720.0,BTC-30JUN23,27440.0,26684.55,1684935327872,26722.5,BTC,26725.0,NaN,NaN
1,165030.0,165030.0,6.049553,USD,-2.1465,18571910,27041.25,27038.31,27000.0,27010.0,BTC-29DEC23,27700.0,26684.55,1684935327872,27035.0,BTC,27047.5,NaN,NaN
2,230885940.0,230885940.0,8566.772411,USD,-2.1932,423973060,26690.25,26690.96,26617.5,26690.0,BTC-PERPETUAL,27379.0,26684.55,1684935327872,26690.0,BTC,26690.5,0.0,0.0
3,152380.0,152380.0,5.561014,USD,-1.9279,2997100,27286.25,27305.80,26950.0,27342.5,BTC-29MAR24,27880.0,26684.55,1684935327872,27265.0,BTC,27307.5,NaN,NaN
4,2889710.0,2889710.0,106.966317,USD,-2.3239,2891940,26705.00,26701.78,26640.0,26690.0,BTC-2JUN23,27427.5,26684.55,1684935327872,26702.5,BTC,26707.5,NaN,NaN
5,6382580.0,6382580.0,236.399133,USD,-2.1988,23232430,26691.25,26689.84,26617.5,26687.5,BTC-26MAY23,27387.5,26684.55,1684935327872,26690.0,BTC,26692.5,NaN,NaN
6,3786130.0,3786130.0,139.200896,USD,-2.1864,60040780,26848.75,26848.43,26780.0,26842.5,BTC-29SEP23,27542.5,26684.55,1684935327872,26847.5,BTC,26850.0,NaN,NaN


# Structured query language (SQL)

[SQL (Structured Query Language)](https://en.wikipedia.org/wiki/SQL) is used for managing and manipulating [relational](https://en.wikipedia.org/wiki/Relational_model) databases. We will explore how to use SQL in Python using Pandas and SQLAlchemy. SQLAlchemy is a SQL toolkit and Object-Relational Mapping (ORM) library that allows Python programs to interact with databases using SQL. There exists several implementations of SQL, we will use [SQLite](https://www.sqlite.org/index.html) as a simple and lightweight example that runs out of the box with our Anaconda distribution.

Please note that this is not an SQL class so we will just scratch the surface. If you have never used SQL before, there exists plenty tutorials on the internet - also consider the links below.

See:
* https://www.sqlite.org/index.html
* https://www.sqlalchemy.org/

In [14]:
from sqlalchemy import create_engine

# Connect to the database
engine = create_engine('sqlite:///slides_pt4.db')

# Write data to the database
market_data.to_sql("market_data", engine, if_exists="replace", index=False)

7

At ots heart, every SQl database is a collection of tables. The code example above connects to or creates (if it doesn't exist) a database called `slides_pt4.db` and saves `market_data` into a table of the same name. 

We can now access the data using a so called SQL query. Think of a query as a filter that selects data from one or multiple tables inside a database. Here are a few code examples that collect data from a table called 'market_data' inside slides_pt4.db.

In [15]:
# Load all data
df = pd.read_sql("""SELECT * FROM market_data""", engine)

display(df)

,volume_usd,volume_notional,volume,quote_currency,price_change,open_interest,mid_price,mark_price,low,last,instrument_name,high,estimated_delivery_price,creation_timestamp,bid_price,base_currency,ask_price,funding_8h,current_funding
0,11430970.0,11430970.0,422.404033,USD,-2.2230,153555650,26723.75,26723.38,26662.5,26720.0,BTC-30JUN23,27440.0,26684.55,1684935327872,26722.5,BTC,26725.0,NaN,NaN
1,165030.0,165030.0,6.049553,USD,-2.1465,18571910,27041.25,27038.31,27000.0,27010.0,BTC-29DEC23,27700.0,26684.55,1684935327872,27035.0,BTC,27047.5,NaN,NaN
2,230885940.0,230885940.0,8566.772411,USD,-2.1932,423973060,26690.25,26690.96,26617.5,26690.0,BTC-PERPETUAL,27379.0,26684.55,1684935327872,26690.0,BTC,26690.5,0.0,0.0
3,152380.0,152380.0,5.561014,USD,-1.9279,2997100,27286.25,27305.80,26950.0,27342.5,BTC-29MAR24,27880.0,26684.55,1684935327872,27265.0,BTC,27307.5,NaN,NaN
4,2889710.0,2889710.0,106.966317,USD,-2.3239,2891940,26705.00,26701.78,26640.0,26690.0,BTC-2JUN23,27427.5,26684.55,1684935327872,26702.5,BTC,26707.5,NaN,NaN
5,6382580.0,6382580.0,236.399133,USD,-2.1988,23232430,26691.25,26689.84,26617.5,26687.5,BTC-26MAY23,27387.5,26684.55,1684935327872,26690.0,BTC,26692.5,NaN,NaN
6,3786130.0,3786130.0,139.200896,USD,-2.1864,60040780,26848.75,26848.43,26780.0,26842.5,BTC-29SEP23,27542.5,26684.55,1684935327872,26847.5,BTC,26850.0,NaN,NaN


In [16]:
# Select data based on some condition
df = pd.read_sql("""
    SELECT
        *
    FROM market_data
    WHERE open_interest > 100000000
    """, engine)

display(df)

,volume_usd,volume_notional,volume,quote_currency,price_change,open_interest,mid_price,mark_price,low,last,instrument_name,high,estimated_delivery_price,creation_timestamp,bid_price,base_currency,ask_price,funding_8h,current_funding
0,11430970.0,11430970.0,422.404033,USD,-2.2230,153555650,26723.75,26723.38,26662.5,26720.0,BTC-30JUN23,27440.0,26684.55,1684935327872,26722.5,BTC,26725.0,NaN,NaN
1,230885940.0,230885940.0,8566.772411,USD,-2.1932,423973060,26690.25,26690.96,26617.5,26690.0,BTC-PERPETUAL,27379.0,26684.55,1684935327872,26690.0,BTC,26690.5,0.0,0.0


In [17]:
# Select specific columns and (optionally) rename columns
df = pd.read_sql("""
    SELECT
        mid_price,
        high,
        low as low_price,  -- renamed column
        last as last_price,  -- renamed column
        instrument_name as ticker  -- renamed column
    FROM market_data
    WHERE open_interest > 100000000
    """, engine)

display(df)

,mid_price,high,low_price,last_price,ticker
0,26723.75,27440.0,26662.5,26720.0,BTC-30JUN23
1,26690.25,27379.0,26617.5,26690.0,BTC-PERPETUAL


This is similar to import functions like `pd.read_csv()`, however, gives us a lot of flexibility in filtering the data. Also, we do not have to load the entire data into [memory (RAM)](https://www.sqlite.org/malloc.html), which allows us to handle very large datasets. If the database is stored on a server, such as [AWS-RDS](https://aws.amazon.com/rds/), the query uses the server's resources before returning only the filtered subset of the data.

# More on Git (.gitignore)

As we are talking about storing data, it makes sense to re-visit Git and GitHub. Please note that there are size limits on the files and the overall repository [(see HERE)](https://docs.github.com/en/repositories/working-with-files/managing-large-files/about-large-files-on-github). Therefore, in general, we do not want to use Git/GitHub as a storage for (large) data files or databases. In order to ensure that we do not accidentally commit files that are not supposed to be versioned, we exclude them using something called a ".gitignore" file.

See:
* https://git-scm.com/docs/gitignore

The .gitignore file is a configuration file used by Git to determine which files and directories should be ignored or excluded from being tracked. It allows you to specify patterns that match certain files or directories, indicating that Git should not consider them for version control. 

The main purpose of the ".gitignore" file is to exclude files that are generated during the development process, contain sensitive information, or are not necessary for the project's functioning. For example, build artifacts, log files, temporary files, and configuration files specific to individual developers or their environments can be safely ignored.

It's worth noting that the ".gitignore" file can be committed to the repository, allowing it to be shared among team members. However, if a file is already being tracked by Git before it's added to the ".gitignore" file, it will continue to be tracked even if it matches the specified patterns. In such cases, you need to explicitly remove the file from Git using the "git rm" command.

If you completed  homework/01_setup.ipynb, you should already have a .gitignore file inside your repository. The first few lines of the .gitignore file that I currently use for this class look like this:

```Bash
# Lecture specific
*.db

# Byte-compiled / optimized / DLL files
__pycache__/
*.py[cod]
*$py.class

# C extensions
*.so

# Distribution / packaging
.Python
build/
develop-eggs/
dist/

# ...
```